In [33]:
import os

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, Subtract, Lambda
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
import keras.backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import cv2
import itertools
import time

In [2]:
labels = {
  "surprise": 0,
  "angry": 1,
  "happy": 2,
  "fear": 3,
  "disgust": 4,
  "sad": 5,
  "neutral": 6
}

In [3]:
def abc(str1):
    for i in range (len(str1)-5,0,-1):
        try:
            z = int(str1[i])
        except:
            return str1[:i+1]

In [17]:
def getImage(im,k):
    im = plt.imread('../Datasets/Facial-Expression-Recognization-using-JAFFE-master/jaffe/AllFiles/'+im.numpy()[k].decode('utf8'))
    im = im.astype('float32')
    im = im/255
    im=cv2.resize(im,(128,128))
    img = im.reshape((-1,128 * 128))
    return img

In [5]:
data_path = '../Datasets/Facial-Expression-Recognization-using-JAFFE-master/id'
data_dir_list = os.listdir(data_path)

n_class = 7
img_data_list=[]
x = []
y = []
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    combinations = list(itertools.combinations(range(len(img_list)), 2))
    for i in range (len(combinations)):
        imgName1 = img_list[combinations[i][0]]
        imgName2 = img_list[combinations[i][1]]
#         label1 = labels[abc(imgName1)]
#         label2 = labels[abc(imgName2)]
        label1 = tf.one_hot(labels[abc(imgName1)], n_class)
        label2 = tf.one_hot(labels[abc(imgName2)], n_class)
#         img1 = plt.imread(data_path + '/'+ dataset + '/'+ imgName1)
#         img2 = plt.imread(data_path + '/'+ dataset + '/'+ imgName2)
#         img1 = cv2.resize(img1,(128,128))
#         img2 = cv2.resize(img2,(128,128))
#         img1 = img1.astype('float32')
#         img1 = img1/255
#         img2 = img2.astype('float32')
#         img2 = img2/255
#         img1 = img1.reshape((-1, 128 * 128))
#         img2 = img2.reshape((-1, 128 * 128))
        x.append((imgName1,imgName2))
        y.append((label1,label2))
        



# img_data = img_data.reshape((-1, 128 * 128))

Loaded the images of dataset-id10

Loaded the images of dataset-id1

Loaded the images of dataset-id4

Loaded the images of dataset-id8

Loaded the images of dataset-id9

Loaded the images of dataset-id6

Loaded the images of dataset-id7

Loaded the images of dataset-id2

Loaded the images of dataset-id5

Loaded the images of dataset-id3



In [39]:
len(x)

2152

In [7]:
# x = x[:-14]

In [8]:
# y = y[:-14]

In [28]:
# Building model
n_class = 7
input_dim = 16384
z_dim = 500



# Encoder
def make_encoder_model():
    inputs = tf.keras.Input(shape=(input_dim,), name='Original_input')
    x = tf.keras.layers.Dense(1024, activation='relu')(inputs)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    latent = tf.keras.layers.Dense(z_dim, activation='linear', name='Latent_variables')(x)
    observed = tf.keras.layers.Dense(n_class, activation='softmax', name='Observed_variables')(x)

    model = tf.keras.Model(inputs=inputs, outputs=[latent, observed], name='Encoder')
    return model


encoder = make_encoder_model()


# # In[ ]:


encoder.summary()

Model: "Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Original_input (InputLayer)     [(None, 16384)]      0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 1024)         16778240    Original_input[0][0]             
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 1024)         1049600     dense_9[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 512)          524800      dense_10[0][0]                   
____________________________________________________________________________________________

In [29]:
# # Decoder
def make_decoder_model():
    inputted_latent = tf.keras.Input(shape=(z_dim,), name='Latent_variables')
    inputted_observed = tf.keras.Input(shape=(n_class,), name='Observed_variables')

    x = tf.keras.layers.concatenate([inputted_latent, inputted_observed], axis=-1)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    reconstruction = tf.keras.layers.Dense(input_dim, activation='linear', name='Reconstruction')(x)
    model = tf.keras.Model(inputs=[inputted_latent, inputted_observed], outputs=reconstruction, name='Decoder')
    return model


decoder = make_decoder_model()

In [31]:
# Multipliers
alpha = 1000.0
beta = 100.0
gamma = 1000.0
zeta = 100

# Loss functions
# Reconstruction cost
mse_loss_fn = tf.keras.losses.MeanSquaredError()

# Supervised cost
cat_loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)


# In[35]:


# Unsupervised cross-covariance cost
def xcov_loss_fn(latent, observed,batch_size):
    latent_centered = latent - tf.reduce_mean(latent, axis=0, keepdims=True)
    observed_centered = observed - tf.reduce_mean(observed, axis=0, keepdims=True)
    xcov_loss = 0.5 * tf.reduce_sum(
        tf.square(tf.matmul(latent_centered, observed_centered, transpose_a=True)))

    return xcov_loss



In [12]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

In [38]:
batch_size = 2
n_epochs = 1

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = train_dataset.batch(batch_size)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)

In [24]:
# Training step
# @tf.function
def train_on_batch(batch_x, batch_y):
    with tf.GradientTape() as tape:
        # Inference
        batch_x_1 = np.zeros((batch_size,128*128))
        batch_x_2 = np.zeros((batch_size,128*128))
        
        batch_y_1 = np.zeros((batch_size,7))
        batch_y_2 = np.zeros((batch_size,7))
        
        for i in range (batch_size):     
            batch_x_1[i] = getImage(batch_x[i],0)
            batch_x_2[i] = getImage(batch_x[i],1)

            batch_y_1[i] = batch_y[i].numpy()[0]
            batch_y_2[i] = batch_y[i].numpy()[1]
            
        batch_latent_1, batch_observed_1 = encoder(batch_x_1)
        batch_latent_2, batch_observed_2 = encoder(batch_x_2)
        batch_reconstruction_1 = decoder([batch_latent_1, batch_observed_1])
        batch_reconstruction_2 = decoder([batch_latent_2, batch_observed_2])
        
#         plt.figure()
#         print(batch_x_1)
#         print(batch_observed_1)
        # Loss functions
        recon_loss_1 = alpha * mse_loss_fn(batch_x_1, batch_reconstruction_1)
        recon_loss_2 = alpha * mse_loss_fn(batch_x_2, batch_reconstruction_2)
#         print(batch_reconstruction_1.shape)
#         plt.imshow(batch_latent_1)
#     plt.imshow(batch_latent_1)
        
        cat_loss_1 = beta * cat_loss_fn(batch_y_1, batch_observed_1)
        cat_loss_2 = beta * cat_loss_fn(batch_y_2, batch_observed_2)
        
        xcov_loss_1 = gamma * xcov_loss_fn(batch_latent_1, batch_observed_1, tf.cast(tf.shape(batch_x_1)[0], tf.float32))
        xcov_loss_2 = gamma * xcov_loss_fn(batch_latent_2, batch_observed_2, tf.cast(tf.shape(batch_x_2)[0], tf.float32))
    
        similarity_loss = zeta*mse_loss_fn(batch_latent_1,batch_latent_2)
        
        # Final loss function
        ae_loss = recon_loss_1 + recon_loss_2 + cat_loss_1 +cat_loss_2 + xcov_loss_1 + xcov_loss_2 + similarity_loss
               
    gradients = tape.gradient(ae_loss, encoder.trainable_variables + decoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, encoder.trainable_variables + decoder.trainable_variables))
    recon_loss = (recon_loss_1 + recon_loss_2)/2
    cat_loss = (cat_loss_1 + cat_loss_2)/2
    xcov_loss = (xcov_loss_1 + xcov_loss_2)/2
    return recon_loss, cat_loss, xcov_loss

In [35]:
for epoch in range(n_epochs):
    start = time.time()
    
    # Functions to calculate epoch's mean performance
    epoch_recon_loss_avg = tf.metrics.Mean()
    epoch_cat_loss_avg = tf.metrics.Mean()
    epoch_xcov_loss_avg = tf.metrics.Mean()

    for batch, (batch_x, batch_y) in enumerate(train_dataset):
        recon_loss, cat_loss, xcov_loss = train_on_batch(batch_x, batch_y)
        epoch_recon_loss_avg(recon_loss)
        epoch_cat_loss_avg(cat_loss)
        epoch_xcov_loss_avg(xcov_loss)
        

    epoch_time = time.time() - start
    print('{:3d}: {:.2f}s ETA: {:.2f}s  Reconstruction cost: {:.4f}  Supervised cost: {:.4f}  XCov cost: {:.4f}'
          .format(epoch + 1, epoch_time,
                  epoch_time * (n_epochs - epoch),
                  epoch_recon_loss_avg.result(),
                  epoch_cat_loss_avg.result(),
                epoch_xcov_loss_avg.result()))

    
   


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Rec1 tf.Tensor(244.77054, shape=(), dtype=float32)
Rec1 tf.Tensor(208.5381, shape=(), dtype=float32)
Rec2 tf.Tensor(0.02782128, shape=(), dtype=float32)
Rec1 tf.Tensor(316.96625, shape=(), dtype=float32)
Rec1 tf.Tensor(197.6529, shape=(), dtype=float32)
Rec2 tf.Tensor(0.016560955, shape=(), dtype=float32)
Rec1 tf.Tensor(206.92264, shape=(), dtype=float32)
Rec1 tf.Tensor(663.6678, shape=(), dtype=float32)
Rec2 tf.Tensor(0.89562964, shape=(), dtype=float32)
Rec1 tf.Tensor(102.310616, shape=(), dtype=float

KeyboardInterrupt: 